# Text tokenization and vectorization

In [1]:
import sys
sys.path.append("..")

import json
from pprint import pprint
import numpy as np
import pandas as pd
import pickle

from src.dataset import Dataset
from src.tokenizers import *
from src.vectorizers import *

## Load processed dataset

In [2]:
dataset = Dataset(dataset_path="../data/raw/data.jsonl", save_path="../data/processed/processed.jsonl")
data = dataset.load_json()

## Create list of texts to tokenize

In [3]:
texts_list = [document["opinions"][i]["text"] for document in data for i in range(len(document["opinions"]))]

## Vectorize text list with Spacy tokenizer

We then save the vectors and vectorizer to disk

In [8]:
vec = TfIdfVectorizer(texts_list[:500], SpacyTokenizer())
vectors = vec.vectors()

In [5]:
vec.save_vectors_vectorizer(vectors)

In [6]:
print(vectors.shape)

(500, 33595)


## Load data from npy and pickle files

In [7]:
loaded_vectors, loaded_vec = TfIdfVectors.load_vectors_vectorizer()

NameError: name 'TfIdfVectors' is not defined

In [ ]:
X = pd.DataFrame(loaded_vectors.toarray(), columns=loaded_vec.get_feature_names())

In [ ]:
X.head(10)